# Graph Classification

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from dataclasses import dataclass
from typing import Optional
import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor
from typing import List, Optional, Tuple, Union
import math
import torch
from torch import Tensor
from torch_geometric.nn.models.mlp import Linear
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import BatchNorm
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_mincut_pool

/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### PROTEINS

In [9]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 700
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="PROTEINS",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=2)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=2)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.7098, Val Acc: 0.5181, Test Acc: 0.5476
Seed: 42, Epoch: 002, Loss: 0.6806, Val Acc: 0.6747, Test Acc: 0.6726
Seed: 42, Epoch: 003, Loss: 0.6612, Val Acc: 0.7108, Test Acc: 0.7321
Seed: 42, Epoch: 004, Loss: 0.6458, Val Acc: 0.6928, Test Acc: 0.7262
Seed: 42, Epoch: 005, Loss: 0.6364, Val Acc: 0.6446, Test Acc: 0.7083
Seed: 42, Epoch: 006, Loss: 0.6294, Val Acc: 0.6325, Test Acc: 0.6905
Seed: 42, Epoch: 007, Loss: 0.6273, Val Acc: 0.6325, Test Acc: 0.6845
Seed: 42, Epoch: 008, Loss: 0.6289, Val Acc: 0.6325, Test Acc: 0.6964
Seed: 42, Epoch: 009, Loss: 0.6280, Val Acc: 0.6325, Test Acc: 0.6845
Seed: 42, Epoch: 010, Loss: 0.6261, Val Acc: 0.6325, Test Acc: 0.6905
Seed: 42, Epoch: 011, Loss: 0.6244, Val Acc: 0.6325, Test Acc: 0.6964
Seed: 42, Epoch: 012, Loss: 0.6233, Val Acc: 0.6506, Test Acc: 0.7024
Seed: 42, Epoch: 013, Loss: 0.6223, Val Acc: 0.6687, Test Acc: 0.7143
Seed: 42, Epoch: 014, Loss: 0.6209, Val Acc: 0.6747, Test Acc: 0.7202
Seed: 42, Epoch: 015

### NCI1

In [10]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 150
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="NCI1",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=2.5)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=2.5)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6923, Val Acc: 0.4838, Test Acc: 0.4619
Seed: 42, Epoch: 002, Loss: 0.6902, Val Acc: 0.4838, Test Acc: 0.4619
Seed: 42, Epoch: 003, Loss: 0.6871, Val Acc: 0.4838, Test Acc: 0.4619
Seed: 42, Epoch: 004, Loss: 0.6820, Val Acc: 0.5065, Test Acc: 0.4749
Seed: 42, Epoch: 005, Loss: 0.6751, Val Acc: 0.5812, Test Acc: 0.5332
Seed: 42, Epoch: 006, Loss: 0.6663, Val Acc: 0.5763, Test Acc: 0.6110
Seed: 42, Epoch: 007, Loss: 0.6567, Val Acc: 0.5974, Test Acc: 0.6175
Seed: 42, Epoch: 008, Loss: 0.6462, Val Acc: 0.6250, Test Acc: 0.6029
Seed: 42, Epoch: 009, Loss: 0.6366, Val Acc: 0.6299, Test Acc: 0.6013
Seed: 42, Epoch: 010, Loss: 0.6300, Val Acc: 0.6347, Test Acc: 0.6045
Seed: 42, Epoch: 011, Loss: 0.6277, Val Acc: 0.6315, Test Acc: 0.6175
Seed: 42, Epoch: 012, Loss: 0.6259, Val Acc: 0.6299, Test Acc: 0.6207
Seed: 42, Epoch: 013, Loss: 0.6227, Val Acc: 0.6299, Test Acc: 0.6143
Seed: 42, Epoch: 014, Loss: 0.6207, Val Acc: 0.6315, Test Acc: 0.6143
Seed: 42, Epoch: 015

### NCI109

In [11]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 150
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="NCI109",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=2.5)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=2.5)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6920, Val Acc: 0.5590, Test Acc: 0.5516
Seed: 42, Epoch: 002, Loss: 0.6896, Val Acc: 0.4847, Test Acc: 0.5048
Seed: 42, Epoch: 003, Loss: 0.6867, Val Acc: 0.4847, Test Acc: 0.5097
Seed: 42, Epoch: 004, Loss: 0.6828, Val Acc: 0.4992, Test Acc: 0.5113
Seed: 42, Epoch: 005, Loss: 0.6778, Val Acc: 0.5234, Test Acc: 0.5323
Seed: 42, Epoch: 006, Loss: 0.6718, Val Acc: 0.5784, Test Acc: 0.5726
Seed: 42, Epoch: 007, Loss: 0.6638, Val Acc: 0.6204, Test Acc: 0.5790
Seed: 42, Epoch: 008, Loss: 0.6561, Val Acc: 0.6511, Test Acc: 0.5968
Seed: 42, Epoch: 009, Loss: 0.6499, Val Acc: 0.6543, Test Acc: 0.6065
Seed: 42, Epoch: 010, Loss: 0.6450, Val Acc: 0.6559, Test Acc: 0.6113
Seed: 42, Epoch: 011, Loss: 0.6399, Val Acc: 0.6478, Test Acc: 0.6274
Seed: 42, Epoch: 012, Loss: 0.6349, Val Acc: 0.6591, Test Acc: 0.6161
Seed: 42, Epoch: 013, Loss: 0.6325, Val Acc: 0.6446, Test Acc: 0.6145
Seed: 42, Epoch: 014, Loss: 0.6281, Val Acc: 0.6624, Test Acc: 0.6258
Seed: 42, Epoch: 015

### MUTAG

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import BaseTransform
from torch_geometric.utils import to_dense_adj
data_path = "/data/XXX/Pooling"
max_nodes = 150
class ConvertToDenseAdj(BaseTransform):
    def __call__(self, data):
        if hasattr(data, 'adj') and data.adj.dim() == 3:
            data.adj = data.adj.sum(dim=-1)
        return data
dataset_dense = TUDataset(
    data_path,
    name="MUTAG",
    transform=T.Compose([T.ToDense(max_nodes), ConvertToDenseAdj()]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
dataset_dense[3]

In [5]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 58.98 seconds
Var Time: 48.53 seconds
Average Memory: 272.67 MB
Average Best Val Acc: 0.9048
Std Best Test Acc: 0.0163
Average Test Acc: 0.8506


### DD

In [6]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
max_nodes = 500
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="DD",
    transform=T.Compose([T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6906, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 002, Loss: 0.6889, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 003, Loss: 0.6877, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 004, Loss: 0.6868, Val Acc: 0.5495, Test Acc: 0.5676
Seed: 42, Epoch: 005, Loss: 0.6861, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 006, Loss: 0.6854, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 007, Loss: 0.6847, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 008, Loss: 0.6840, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 009, Loss: 0.6831, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 010, Loss: 0.6820, Val Acc: 0.5495, Test Acc: 0.5586
Seed: 42, Epoch: 011, Loss: 0.6808, Val Acc: 0.5766, Test Acc: 0.5766
Seed: 42, Epoch: 012, Loss: 0.6793, Val Acc: 0.6937, Test Acc: 0.6667
Seed: 42, Epoch: 013, Loss: 0.6775, Val Acc: 0.7297, Test Acc: 0.6937
Seed: 42, Epoch: 014, Loss: 0.6752, Val Acc: 0.7748, Test Acc: 0.7387
Seed: 42, Epoch: 015

### IMDB-BINARY

In [12]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="IMDB-BINARY",
    transform=T.Compose([T.OneHotDegree(136), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=2)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=2)
        x = self.gnn3_embed(x, adj)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 0.6957, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 002, Loss: 0.6939, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 003, Loss: 0.6920, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 004, Loss: 0.6896, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 005, Loss: 0.6866, Val Acc: 0.5200, Test Acc: 0.5267
Seed: 42, Epoch: 006, Loss: 0.6823, Val Acc: 0.5200, Test Acc: 0.5400
Seed: 42, Epoch: 007, Loss: 0.6769, Val Acc: 0.5267, Test Acc: 0.5933
Seed: 42, Epoch: 008, Loss: 0.6697, Val Acc: 0.6533, Test Acc: 0.7400
Seed: 42, Epoch: 009, Loss: 0.6613, Val Acc: 0.7200, Test Acc: 0.8000
Seed: 42, Epoch: 010, Loss: 0.6513, Val Acc: 0.7867, Test Acc: 0.7933
Seed: 42, Epoch: 011, Loss: 0.6391, Val Acc: 0.7533, Test Acc: 0.8000
Seed: 42, Epoch: 012, Loss: 0.6250, Val Acc: 0.7467, Test Acc: 0.7667
Seed: 42, Epoch: 013, Loss: 0.6105, Val Acc: 0.7533, Test Acc: 0.7800
Seed: 42, Epoch: 014, Loss: 0.5928, Val Acc: 0.7467, Test Acc: 0.7800
Seed: 42, Epoch: 015

### IMDB-MULTI

In [13]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="IMDB-MULTI",
    transform=T.Compose([T.OneHotDegree(88), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=1.5)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=1.5)
        x = self.gnn3_embed(x, adj)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 709.59 seconds
Var Time: 389.41 seconds
Average Memory: 4620.00 MB
Average Best Val Acc: 0.5289
Std Best Test Acc: 0.0255
Average Test Acc: 0.4593


### COLLAB

In [14]:
max_nodes = 500
data_path = "/data/XXX/Pooling"
dataset_dense = TUDataset(
    data_path,
    name="COLLAB",
    transform=T.Compose([T.OneHotDegree(491), T.ToDense(max_nodes)]),
    use_node_attr=True,
    pre_filter=lambda data: data.num_nodes <= max_nodes,
)
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
import random
from torch_geometric.nn import GCNConv
import os.path as osp
import time
from math import ceil
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, normalize=False, lin=True):
        super().__init__()
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)
        if lin:
            self.lin = torch.nn.Linear(out_channels, out_channels)
        else:
            self.lin = None
    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()
        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x
    def forward(self, x, adj, mask=None):
        x = self.bn(1, self.conv1(x, adj, mask).relu())
        x = self.bn(2, self.conv2(x, adj, mask).relu())
        x = self.bn(3, self.conv3(x, adj, mask).relu())
        return x
class Net_mincut(torch.nn.Module):
    def __init__(self):
        super().__init__()
        num_nodes = 64
        self.gnn1_pool = GNN(dataset_dense.num_features, 64, num_nodes)
        self.gnn1_embed = DenseGCNConv(dataset_dense.num_features, 64)
        num_nodes = 64
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = DenseGCNConv(64, 64)
        self.gnn3_embed = DenseGCNConv(64, 64)
        self.lin1 = torch.nn.Linear(64, 32)
        self.lin2 = torch.nn.Linear(32, dataset_dense.num_classes)
    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x = F.relu(x)
        x, adj, l1, e1 = dense_mincut_pool(x, adj, s, mask, temp=1.5)
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)
        x = F.relu(x)
        x, adj, l2, e2 = dense_mincut_pool(x, adj, s, temp=1.5)
        x = self.gnn3_embed(x, adj)
        x = F.relu(x)
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = Net_mincut().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        total_loss += data.y.size(0) * float(loss)
        optimizer.step()
    return total_loss / len(train_loader.dataset)
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        output, _, _ = model(data.x, data.adj, data.mask)
        pred = output.max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []
early_stop_patience = 150
tolerance = 0.0001
for seed in seeds:
    set_seed(seed)
    dataset_dense = dataset_dense.shuffle()
    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    num_total = len(dataset_dense)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_dense[:num_train]
    val_dataset = dataset_dense[num_train:num_train + num_val]
    test_dataset = dataset_dense[num_train + num_val:]
    train_loader = DenseDataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DenseDataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DenseDataLoader(test_dataset, batch_size=512, shuffle=False)
    model = Net_mincut().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')
        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break
    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  
    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)
    torch.cuda.empty_cache()
print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.1241, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 002, Loss: 1.1040, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 003, Loss: 1.0642, Val Acc: 0.3213, Test Acc: 0.3320
Seed: 42, Epoch: 004, Loss: 0.9924, Val Acc: 0.4973, Test Acc: 0.5107
Seed: 42, Epoch: 005, Loss: 0.8994, Val Acc: 0.6440, Test Acc: 0.6453
Seed: 42, Epoch: 006, Loss: 0.8042, Val Acc: 0.6107, Test Acc: 0.6293
Seed: 42, Epoch: 007, Loss: 0.7230, Val Acc: 0.6267, Test Acc: 0.6400
Seed: 42, Epoch: 008, Loss: 0.6463, Val Acc: 0.6400, Test Acc: 0.6453
Seed: 42, Epoch: 009, Loss: 0.5924, Val Acc: 0.6573, Test Acc: 0.6573
Seed: 42, Epoch: 010, Loss: 0.5632, Val Acc: 0.6800, Test Acc: 0.6840
Seed: 42, Epoch: 011, Loss: 0.5401, Val Acc: 0.6973, Test Acc: 0.6853
Seed: 42, Epoch: 012, Loss: 0.5226, Val Acc: 0.7053, Test Acc: 0.6933
Seed: 42, Epoch: 013, Loss: 0.5043, Val Acc: 0.7173, Test Acc: 0.6973
Seed: 42, Epoch: 014, Loss: 0.4910, Val Acc: 0.7227, Test Acc: 0.7027
Seed: 42, Epoch: 015

# Graph Regression

### QM7

In [5]:
!python /data/XXX/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num 0 --run_times=5 --patience=150 --epochs=500 --pooling='MinCut'

Existed dataset loaded: datasets/processed/qm7.pt

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500Traceback (most recent call last):
  File "/data/XXX/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py", line 140, in <module>
    _, metric_te = trainer.fit_and_test(loader_tr[i], loader_va[i], loader_te[i], log_train_results=args.log_train_results,
  File "/data/XXX/Pooling/Graph_Pooling_Benchmark/Regression/process/trainer.py", line 166, in _train_test_reg
    val_loss, val_metric = self._eval_reg(loader_va)
  File "/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/data/XXX/Pooling/Graph_Pooling_Benchmark/Regression/process/trainer.py", line 337, in _eval_reg
    val = mean_absolute_error(y_true, y_scores) # , multioutput='raw_values'
  File "/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/sklearn/